In [70]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd

#Directory containing the RF scan data
dataFolder = 'C:/Users/Lucas/OneDrive - The University of Western Ontario/Documents/GitHub/SE4450-project-code/Preprocessing/P82-W0-S2.mat'

#Loading in the matlab file information
rawMatFile = loadmat(dataFolder)

#Contains just the RF data pulled from the .mat file
#RFData = [[element for element in upperElement] for upperElement in rawMatFile['rf1']]
RFData = rawMatFile['rf1']
print(type(RFData))
print(type(rawMatFile['rf1']))

#Kinda works to leave a slice of each edge of the scan from being black
cutTop = 0.1
cutLeft = 0.1
cutBottom = 0.1
cutRight = 0.1

lowerI = int(cutLeft*256)
upperI = int((1-cutRight)*256)
lowerJ = int(cutTop*2592)
upperJ = int((1-cutBottom)*2592)

frames = [] #vector to hold individual cropped frames

for x in range(RFData.shape[2]): #range is how many frames there are in the set
    frames.append(RFData[lowerJ:upperJ,lowerI:upperI,x]) #Take inner subset of frame to allow for translation

pairs = generatePairs(frames)
extra = translatePair(pairs[2],1)
#print(len(extra))
#print(extra[0][0].shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
9
(2073, 205)


In [ ]:
def generatePairs(frames):
    framePairs = [] #vector to hold image pairs

    for x in range(len(frames)): #x represents the first frame in the pair
        for y in range(len(frames)-5-x): #y represents second frame in the pair
            pair = [(x+1),(x+y+6),frames[x],frames[x+y+5]] #generate apir
            framePairs.append(pair)
    return framePairs

In [61]:
def translatePair(pair,steps):

    frame1 = pair[0]-1 #get first frame position from pair object
    frame2 = pair[1]-1 #get second frame position from pair object
    JAdd = pair[2].shape[0] #Get size of original frame to keep translation size consistent
    IAdd = pair[2].shape[1] #Get size of original frame to keep translation size consistent

    baseCut = 0.1 #use same base cut as is used in original frame generation
    step = baseCut/steps #set step size to be as large as possible for given number of steps
    pairs = [] #vector to hold newly generated pairs

    #adjust how the frame is cropped from the original data to simulate translation
    for x in range(-steps,steps+1): 
        for y in range(-steps,steps+1):
            lowerI = int((cutLeft+x*step)*256)
            upperI = lowerI+IAdd
            lowerJ = int((cutTop+y*step)*2592)
            upperJ = lowerJ+JAdd

            pairs.append([RFData[lowerJ:upperJ,lowerI:upperI,frame1],RFData[lowerJ:upperJ,lowerI:upperI,frame2]]) #append translated pair to pairs vector
    return pairs

In [ ]:
import re #for regular expressions to allow sorting the frames

def numberRegex(e):
  #Finding all digits in the filename
  x = re.findall("(\d)", e)

  #if there are two digits in filename
  if(len(x)==2):
      return int(x[0]+""+x[1])
  #if there is one digit in the filename
  else:
      return int(x[0])


In [11]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd

#Testing to see if we can make dataframe straight from the raw mat file dictionary
dataFolder = 'H:/Documents/Github repositories/SE4450-project-code/Preprocessing/P82-W0-S2.mat'
rawMatFile = loadmat(dataFolder)
print(rawMatFile['rf1'].shape[2])


19


In [32]:
from scipy.io import loadmat
from scipy.io import savemat
import pandas as pd

#Lucas directory
#C:/Users/Lucas/OneDrive - The University of Western Ontario/Documents/GitHub/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

#Ian directory
#H:/Documents/Github repositories/SE4450-project-code/Preprocessing/Pxx-Wx-Sx_RF

#Directory containing the RF scan data
dataFolder = 'H:/Documents/Github repositories/SE4450-project-code/Preprocessing/P82-W0-S2.mat'
#vecRFDataNames = os.listdir(os.path.join(dataFolder)) #vector with the names of the images in the above folder
#vecRFDataNames.sort(key=numberRegex) #Sorts the images by frame number

#Loading in the matlab file information
rawMatFile = loadmat(dataFolder)

#Contains just the RF data pulled from the .mat file
#RFData = [[element for element in upperElement] for upperElement in rawMatFile['rf1']]
RFData = rawMatFile['rf1']
print(type(RFData))
print(type(rawMatFile['rf1']))
#Kinda works to leave a slice of each edge of the scan from being black
cutTop = 0.1
cutLeft = 0.1
cutBottom = 0.1
cutRight = 0.1

lowerI = int(cutLeft*256)
upperI = int((1-cutRight)*256)
lowerJ = int(cutTop*2592)
upperJ = int((1-cutBottom)*2592)

#zeroArray = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(256):
    for j in range(2592):
        if j>lowerJ and j<upperJ: #rows to zero out
            if i>lowerI and i<upperI: #cols to zero out
                RFData[j][i][0] = 0 #the 0 in the third [] pertains to the frame to edit (0 being frame 1)
 
# Updates inputted matlab file to contain changes from the for loop above
rawMatFile['rf1'] = RFData

# Saves file while applying changes from the for loop above
# matDic = {'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Oct 21 17:26:33 2020','__version__':'1.0', '__globals__':'','rf1':RFData}
# savemat("transformTest.mat", matDic, long_field_names=True)

# Saves the mat file without any changes
savemat("transformTestUnchanged.mat", rawMatFile)

#Converting python dictionary into pandas dataframe for easy manipulation (unsure if needed)
#RFDataframe = pd.DataFrame(rawMatFile['rf1'])

#print(RFDataframe.head(5))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
105


In [ ]:
#This was a long shot to visualize the RF dataframe
import matplotlib.pyplot as plt
import numpy as np

[m,n] = np.shape(RFData[0:10][0:10])

#Colour Map using Matrix
plt.figure()
plt.imshow(RFData, alpha=0.8)
plt.xticks(np.arange(n))
plt.yticks(np.arange(m))
plt.xlabel('Numbers')
plt.ylabel('Value')
plt.title('Color Maps')